<a href="https://colab.research.google.com/github/prxshxntray/cnn-biomass-regression/blob/main/CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
print(torch.xpu.is_available())  # torch.xpu is the API for Intel GPU support
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms
import pathlib
import os
from PIL import Image
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm

from cnn_workflow_utils import (
    ImageDataset,
    TARGET_COLUMNS,
    build_model,
    calculate_global_weighted_r2,
    get_device,
    train_regression,
)


False


ModuleNotFoundError: No module named 'cnn_workflow_utils'

Packages, take note that line 6 is specific to my Intel Core GPU, please change (and in subsequent code below) to suit your own GPU [Also downloading Pytorch on different GPU setups might be an issue since I assume most will be using Nvidea - of which I am not using ; so please amend ]

**LOADING DATA**

In [ ]:
df = pd.read_csv('../data/train.csv')
df.head()

I have saved all of the datasets under ../data/ though I am not sure how this is replicated on google colab - need to check

In [ ]:
df_test = pd.read_csv('../data/test.csv')
df_test.head()

I loaded in the test dataset but I did not run my code on it fyi, incase you were wondering

**DATA MANIPULATION**

In [ ]:
df_wide = df.copy()
df_wide = df_wide.pivot(index = 'image_path', columns = 'target_name', values = 'target').reset_index()
df_wide["cleaned_path"] = df_wide["image_path"].str.replace(r'^train[\\/]', '', regex=True)
df_wide.head()

**SET UP FOR CNN / FUNCTIONS FOR CNN**

In [ ]:
image_folder = '../data/train'
assert os.path.exists(image_folder), f"Image folder {image_folder} does not exist."

df_wide["full_path"] = df_wide["cleaned_path"].apply(
    lambda fname: os.path.join(image_folder, fname)
)

df_wide.head(7)

This merges the picture directory to the excel table

In [ ]:
image_folder_test = '../data/test'
assert os.path.exists(image_folder_test), f"Image folder {image_folder_test} does not exist."


Random check

In [ ]:
test_path = "../data/train\\ID1011485656.jpg"
print("Path:", test_path)
print("Exists?", os.path.exists(test_path))

In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df_wide, test_size=0.3, random_state=1)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=1)

Train test Val split

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),        # ← better than CenterCrop for training
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # ← adds variety
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

eval_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),        # ← deterministic for val/test
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Tranformation set up for the pictures; to be pre processed before feeding into CNN

In [ ]:
# ImageDataset is imported from cnn_workflow_utils.


Function to Transform the excel table data and picture into meaningful Tensor

In [ ]:
train_dataset = ImageDataset(train_df, transform = train_transform)
val_dataset = ImageDataset(val_df, transform=eval_transform)
test_dataset = ImageDataset(test_df, transform=eval_transform)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

DataLoader for the CNN

In [ ]:
# Get one batch
for images, targets in train_loader:
    print("Image batch shape:", images.shape)      # Should be [batch_size, 3, 224, 224]
    print("Target batch shape:", targets.shape)   # Should be [batch_size, 5]
    print("Image dtype:", images.dtype)           # Should be torch.float32
    print("Target sample:", targets[0])           # Should be 5 numbers
    break

TEST

In [ ]:
from torchvision import models

weights = models.ResNet18_Weights.IMAGENET1K_V1
model = build_model("resnet18", num_targets=5, weights=weights)

# Swap architecture with 'densenet121' or 'efficientnet_b0' to experiment.
device = get_device()
model = model.to(device)


**IMPORTANT** - seting up the initial RESNET (need to downalod first, not sure how this is reflected in the collab though) Also in line 9, you will need to amend this for sure since it uses my Intel XPU GPU!

In [ ]:
# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze ONLY the final layer
for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.MSELoss()

optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)
# above only fine tunes the Final layer, bottom fine-tunes the entire network

# optimizer = optim.Adam(model.parameters(), lr=1e-4)

Freezes learning for all parameters except the last layer - since the first layer are the pre-trained RESNET.
Used MSE as the Loss function
Used Adam as the Optimisation function

In [ ]:
# # Get a sample batch
# sample_images, sample_targets = next(iter(train_loader))
# sample_images, sample_targets = sample_images.to(device), sample_targets.to(device)

# # Forward pass
# outputs = model(sample_images)
# print("Model output shape:", outputs.shape)  # Should be [batch_size, 5]
# print("Sample prediction:", outputs[0].detach().cpu().numpy())
# print("Sample target:", sample_targets[0].cpu().numpy())

In [ ]:
# train_regression is imported from cnn_workflow_utils.


Function for the regression model

In [ ]:
num_epochs = 20
train_losses, val_losses = train_regression(
    model, num_epochs, train_loader, val_loader,
    criterion, optimizer, device
)

In [ ]:
torch.save(model.state_dict(), "final_model.pth")
print("Model saved to final_model.pth")

Saving the model parameters for easy replication in the future

In [ ]:
# calculate_global_weighted_r2 is imported from cnn_workflow_utils.


Code to find the R^2 as detailed per the Kaggle page

In [ ]:
# Recreate the model architecture
model_eval = build_model("resnet18", num_targets=5, weights=None)

# Load weights
model_eval.load_state_dict(torch.load("final_model.pth", map_location=device))
model_eval = model_eval.to(device)
model_eval.eval()  # set to evaluation mode

# Now evaluate on test set
test_loss = 0.0
all_preds = []      # ← collect all predictions
all_targets = []    # ← collect all targets
with torch.no_grad():
    for images, targets in test_loader:
        images, targets = images.to(device), targets.to(device)
        preds = model_eval(images)
        loss = criterion(preds, targets)
        test_loss += loss.item()
        all_preds.append(preds)
        all_targets.append(targets)

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss (from saved model): {avg_test_loss:.6f}")

# Concatenate all batches into single tensors
all_preds = torch.cat(all_preds, dim=0)      # [N_test, 5]
all_targets = torch.cat(all_targets, dim=0)  # [N_test, 5]
print(all_preds.shape, all_targets.shape)

# Calculate globally weighted R²
test_r2 = calculate_global_weighted_r2(all_preds, all_targets)
print(f"Test Weighted R²: {test_r2:.4f}")


First simple run thus far

In [ ]:
!git clone https://github.com/prxshxntray/cnn-biomass-regression.git